In [59]:
import awkward as ak
import numpy as np
import uproot as up
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib.pyplot as plt
import correctionlib
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.lookup_tools import extractor
from coffea.analysis_tools import PackedSelection

fname = "/home/saikat/analysis/MC_files/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8/ZG2JtoG2L2J_EWK_MLL-50_MJJ-120_TuneCP5_withDipoleRecoil_13p6TeV_madgraph-pythia8.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "ZGammaToLLGamma"},
).events()
totGenWeight = np.sum(events.genWeight)
totGenWeight

698608.0

# Working with $\mu$s

In [60]:
#Applying Trigger cuts 
hlt = events.HLT
#hlt.fields
trig_cut = np.stack((hlt.IsoMu24, hlt.IsoMu27), axis=1)
events = events[ak.any(trig_cut, axis=1)]
events

<NanoEventsArray [<event 1:132:91179>, ... ] type='204527 * event'>

In [61]:
muon = events.Muon
#muon.fields
muon_sel = (ak.num(events.Muon)==2) & (abs(events.Muon.eta)<2.4) & (events.Muon.tightId==1) & (abs(events.Muon.dxy)<0.045) & (abs(events.Muon.dz)<0.2) & (ak.sum(events.Muon.charge, axis=1) == 0)
events.Muon = events.Muon[muon_sel]
#muon = muon[ak.num(muon)==2]
events = events[ak.num(events.Muon)==2]
events.Muon.pt

<Array [[134, 9.84], ... [47.7, 34.4]] type='119868 * var * float32[parameters={...'>

In [62]:
#Di-muon Invariant mass cut
mMuMu_cut = ((events.Muon[:,0]+events.Muon[:,1]).mass< 110) & ((events.Muon[:,0]+events.Muon[:,1]).mass> 70)
events = events[mMuMu_cut]
print(events.Muon)
print((events.Muon[:,0]+events.Muon[:,1]).mass)
print(mMuMu_cut)
events.Muon.pt

[[Muon, Muon], [Muon, Muon], [Muon, Muon, ... Muon], [Muon, Muon], [Muon, Muon]]
[92.5, 88.1, 89.8, 92.1, 72.1, 84.4, 83.2, ... 90.7, 91.5, 96, 88.6, 91.3, 79.8, 90]
[True, True, False, True, False, False, True, ... True, True, True, True, True, True]


<Array [[134, 9.84], ... [47.7, 34.4]] type='91463 * var * float32[parameters={"...'>

In [63]:
(events.Muon[:,0]+events.Muon[:,1]).mass
#events.Photon.pt

<Array [92.5, 88.1, 89.8, ... 91.3, 79.8, 90] type='91463 * float32'>

# Working with $\gamma$

In [64]:
#Applying eta and other quality cuts
photon_sel = (abs(events.Photon.eta)<2.5) & (events.Photon.pixelSeed==0) & (events.Photon.pfPhoIso03<0.8) & (events.Muon[:,0].delta_r(events.Photon)>0.4) & (events.Muon[:,1].delta_r(events.Photon)>0.4)
print(photon_sel)
events.Photon = events.Photon[photon_sel]
events = events[ak.num(events.Photon)==1]

[[False], [True], [True], [True], ... True], [True], [False, False], [True, True]]


In [65]:
# Load the correctionlib JSON file #
evaluator = correctionlib.CorrectionSet.from_file("../EGmSFs/SS.json")
print(list(evaluator.keys()))
# Obtain the smearing values
evaluator_smearing = evaluator["Prompt2022FG_SmearingJSON"]
rho = evaluator_smearing.evaluate("rho", events.Photon[:,0].eta, events.Photon[:,0].r9)
rng = np.random.default_rng(seed=125)  # The smearing is done statistically, so we need some random numbers
smearing = rng.normal(loc=1., scale=rho)
photon_pt_smearing_nom = smearing * events.Photon[:,0].pt

['Prompt2022FG_ScaleJSON', 'Prompt2022FG_SmearingJSON']


In [66]:
file = up.recreate('output_ZGammaToMuMuGamma.root')
#file["tree3"] = {"Photon_pt": sel_photon_pt_smearing_nom,"nJet": nJet,"isJet":jet_pt_eta_cut,"Jet_pt": jet.pt}
file["tree"] = {"lead_muon": ak.zip({
                        "pt": events.Muon[:,0].pt,
                        "eta": events.Muon[:,0].eta,
                        "phi": events.Muon[:,0].phi,
                        "mass":events.Muon[:,0].mass,
                        }),
                "sublead_muon": ak.zip({
                        "pt": events.Muon[:,1].pt,
                        "eta": events.Muon[:,1].eta,
                        "phi": events.Muon[:,1].phi,
                        "mass":events.Muon[:,1].mass,
                        }),
                 
                 "dimuon_inv_mass":(events.Muon[:,0]+events.Muon[:,1]).mass,
                 
                # "Photon": ak.zip({
                #    "pt":photon[:,0].pt,
                #     #"pt_smearing_up":sel_photon_pt_smearing_up,
                #    #"pt_smearing_down":sel_photon_pt_smearing_down,
                #    #"pt_scale_up":sel_photon_pt_scale_up,
                #    #"pt_scale_down":sel_photon_pt_scale_down,
                #    "eta":photon[:,0].eta,
                #    "phi":photon[:,0].phi,
                #})
                }